#### Produce Constructor Standings

In [0]:
dbutils.widgets.text("p_file_date","2021-03-28")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../Includes/Configuration"

In [0]:
%run "../Includes/Common Functions"

Find race years for which the data is to be re-processed

In [0]:
race_results_df = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
    .filter(f"file_date = '{v_file_date}'")

In [0]:
race_year_list = df_column_to_list(race_results_df,'race_year')

In [0]:
from pyspark.sql.functions import col

race_results_df = spark.read.format("delta").load(f"{presentation_folder_path}/race_results") \
    .filter(col("race_year").isin(race_year_list))

In [0]:
from pyspark.sql.functions import sum , when, col, count

constructor_standing_df = race_results_df \
    .groupBy("race_year","team") \
    .agg(sum("points").alias("total_points"),
         count(when(col("position") == 1, True)).alias("wins")) 

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, rank

constructor_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"),desc("wins"))
final_df = constructor_standing_df.withColumn("rank", rank().over(constructor_rank_spec))

In [0]:
#final_df.write.mode("overwrite").parquet(f"{presentation_folder_path}/constructor_standings")
#final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_presentation.constructor_standings")
#overwrite_partition(final_df,'f1_presentation','constructor_standings','race_year')

In [0]:
merge_condition = "tgt.team = src.team AND tgt.race_year = src.race_year"
merge_delta_table(final_df, 'f1_presentation', 'constructor_standings', presentation_folder_path, merge_condition, 'race_year')